In [1]:
#数据分析模块
import pandas as pd
import numpy as np
import random as rnd
from datetime import datetime
import time
import math
#可视化模块
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
plt.rcParams['font.sans-serif'] = ['SimHei'] # 显示中文标题 
plt.rcParams['axes.unicode_minus'] = False # 正常显示正负号 
%matplotlib inline


#模型训练模块
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
#导入数据
df = pd.read_excel('COMAP_RollerCoasterData_2019补充后.xlsx')

In [3]:
# 拿要处理的列
new_df = df[['使用年长','Type','Height (feet)','Length (feet)','Number of Inversions','Drop (feet)','G Force','Vertical Angle (degrees)']]
# 速度这一列搞不了，单独处理了
sp = df.iloc[:,12:13]
new_df['Speed'] = sp.iloc[:,0:1]

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [4]:
# 处理时间，转换为秒
idx = df['Duration (min:sec)'][df['Duration (min:sec)'].isnull()==False].index
for i in idx:
    if i <=143:
        df['Duration (min:sec)'][i] = df['Duration (min:sec)'][i].hour*60 +  df['Duration (min:sec)'][i].minute
    if i > 144:
        df['Duration (min:sec)'][i] = df['Duration (min:sec)'][i].hour*60 +  df['Duration (min:sec)'][i].minute
new_df['Time'] = df['Duration (min:sec)']
new_df = new_df.drop([144])

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
new_df['Type'] = new_df['Type'].map({'Sit Down':0, 'Inverted':1, 'Wing':2, 'Steel':3, 'Stand Up':4, 'Flying':5,
       'Wood':6, 'Suspended':7}).astype(float)

In [6]:
# 删除 Height (feet) 和 Length (feet)的空值行
idx = new_df['Height (feet)'][new_df['Height (feet)'].isnull()==True].index
new_df = new_df.drop(idx)
idx = new_df['Length (feet)'][new_df['Length (feet)'].isnull()==True].index
new_df = new_df.drop(idx)
idx = new_df['使用年长'][new_df['使用年长'].isnull()==True].index
new_df = new_df.drop(idx)
idx = new_df['Speed'][new_df['Speed'].isnull()==True].index
new_df = new_df.drop(idx)

In [7]:
for i in range(0,240):
    print(str(i)+" : "+ str(new_df[['Drop (feet)']].values[i]))
print(new_df[['Drop (feet)']].values[238])
new_df = new_df.drop(new_df[['Drop (feet)']].index[238])
new_df['Drop (feet)'] = new_df[['Drop (feet)']].astype(float)

0 : [104]
1 : [nan]
2 : [nan]
3 : [nan]
4 : [nan]
5 : [nan]
6 : [nan]
7 : [nan]
8 : [nan]
9 : [nan]
10 : [98.4]
11 : [nan]
12 : [nan]
13 : [64]
14 : [nan]
15 : [nan]
16 : [nan]
17 : [nan]
18 : [80]
19 : [80.5]
20 : [nan]
21 : [nan]
22 : [88.6]
23 : [nan]
24 : [105]
25 : [nan]
26 : [nan]
27 : [98]
28 : [nan]
29 : [nan]
30 : [nan]
31 : [nan]
32 : [nan]
33 : [nan]
34 : [90]
35 : [nan]
36 : [90]
37 : [nan]
38 : [90]
39 : [91]
40 : [90]
41 : [nan]
42 : [nan]
43 : [nan]
44 : [nan]
45 : [nan]
46 : [nan]
47 : [80]
48 : [100]
49 : [100]
50 : [nan]
51 : [162]
52 : [81.2]
53 : [nan]
54 : [nan]
55 : [nan]
56 : [nan]
57 : [nan]
58 : [nan]
59 : [nan]
60 : [nan]
61 : [nan]
62 : [nan]
63 : [nan]
64 : [nan]
65 : [nan]
66 : [nan]
67 : [nan]
68 : [nan]
69 : [104.9]
70 : [nan]
71 : [nan]
72 : [nan]
73 : [nan]
74 : [nan]
75 : [nan]
76 : [nan]
77 : [nan]
78 : [nan]
79 : [nan]
80 : [nan]
81 : [nan]
82 : [nan]
83 : [nan]
84 : [nan]
85 : [nan]
86 : [nan]
87 : [nan]
88 : [111.8]
89 : [nan]
90 : [108]
91 : [54]


In [8]:
# 随机森林 预测缺失值,List列表第一个表示要预测的缺失列，其他是原始数据
def getNum_rfr(df,List,Type='float'):
    # 把已有的数值型特征取出来丢进Random Forest Regressor中
    feet_df = df[List]

    # 乘客分成已知年龄和未知年龄两部分
    known_feet = feet_df[feet_df[List[0]].notnull()].as_matrix()
    unknown_feet = feet_df[feet_df[List[0]].isnull()].as_matrix()

    # y即目标年龄
    y = known_feet[:, 0]

    # X即特征属性值
    X = known_feet[:, 1:]

    # fit到RandomForestRegressor之中
    rfr = RandomForestClassifier(n_estimators=100)

    # 导入归一化模块
    from sklearn.preprocessing import StandardScaler
    # 使用该类
    standarScaler = StandardScaler()
    # 传入训练数据作为标准
    standarScaler.fit(X)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # 获得归一化的训练数据
    X_train = standarScaler.transform(X)
    # 获得归一化的测试数据
    X_test_standar = standarScaler.transform(unknown_feet[:, 1:])

    # 只能int类型！！!
    rfr.fit(X_train, y.astype(Type))

    # 用得到的模型进行未知年龄结果预测
    predictedAges = rfr.predict(X_test_standar)
    #  print predictedAges
    # 用得到的预测结果填补原缺失数据
    df.loc[ (df[List[0]].isnull()), List[0]] = predictedAges
    return df

# 决策树
def getNum(df,List,Type='float'):
    feet_df = df[List]

    # 乘客分成已知年龄和未知年龄两部分
    known_feet = feet_df[feet_df[List[0]].notnull()].as_matrix()
    unknown_feet = feet_df[feet_df[List[0]].isnull()].as_matrix()

    # y即目标年龄
    y = known_feet[:, 0]

    # X即特征属性值
    X = known_feet[:, 1:]


    # 导入归一化模块
    from sklearn.preprocessing import StandardScaler
    # 使用该类
    standarScaler = StandardScaler()
    # 传入训练数据作为标准
    standarScaler.fit(X)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # 获得归一化的训练数据
    X_train = standarScaler.transform(X)
    # 获得归一化的测试数据
    X_test_standar = standarScaler.transform(unknown_feet[:, 1:])

    # fit到RandomForestRegressor之中
    decision_tree = DecisionTreeClassifier()

    decision_tree.fit(X_train, y.astype(Type))

    # 用得到的模型进行未知年龄结果预测
    predictedAges = decision_tree.predict(X_test_standar)
    #  print predictedAges
    # 用得到的预测结果填补原缺失数据
    df.loc[ (df[List[0]].isnull()), List[0]] = predictedAges
    return df

In [9]:
List = ['Drop (feet)','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
new_df = getNum_rfr(new_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [10]:
List = ['Time','使用年长', 'Type', 'Height (feet)', 'Length (feet)']
new_df = getNum(new_df,List)

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
List = ['Vertical Angle (degrees)','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
new_df = getNum_rfr(new_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
# 删除空格
for i in range(0,291):
    print(str(i)+" : "+ str(new_df[['G Force']].values[i]))
print(new_df[['G Force']].values[284])
new_df = new_df.drop(new_df[['G Force']].index[284])
new_df['G Force'] = new_df[['G Force']].astype(float)

0 : [3.5]
1 : [4]
2 : [nan]
3 : [nan]
4 : [nan]
5 : [nan]
6 : [nan]
7 : [nan]
8 : [nan]
9 : [nan]
10 : [4.5]
11 : [4.5]
12 : [4.5]
13 : [nan]
14 : [nan]
15 : [nan]
16 : [4.5]
17 : [5]
18 : [nan]
19 : [nan]
20 : [nan]
21 : [nan]
22 : [4]
23 : [nan]
24 : [nan]
25 : [4.5]
26 : [nan]
27 : [3]
28 : [nan]
29 : [nan]
30 : [nan]
31 : [3.8]
32 : [nan]
33 : [nan]
34 : [nan]
35 : [nan]
36 : [nan]
37 : [nan]
38 : [nan]
39 : [nan]
40 : [nan]
41 : [4.9]
42 : [4]
43 : [4]
44 : [4]
45 : [nan]
46 : [nan]
47 : [nan]
48 : [nan]
49 : [nan]
50 : [nan]
51 : [nan]
52 : [4.6]
53 : [nan]
54 : [nan]
55 : [nan]
56 : [nan]
57 : [nan]
58 : [nan]
59 : [nan]
60 : [nan]
61 : [nan]
62 : [nan]
63 : [nan]
64 : [nan]
65 : [nan]
66 : [nan]
67 : [nan]
68 : [nan]
69 : [4]
70 : [4.7]
71 : [4.9]
72 : [nan]
73 : [4.3]
74 : [4.3]
75 : [nan]
76 : [nan]
77 : [5.2]
78 : [5.2]
79 : [5.2]
80 : [5.2]
81 : [5.2]
82 : [5.2]
83 : [5.2]
84 : [5.2]
85 : [5.2]
86 : [5.2]
87 : [5.2]
88 : [4.7]
89 : [nan]
90 : [nan]
91 : [nan]
92 : [nan]
93 

In [13]:
List = ['G Force','使用年长', 'Type', 'Height (feet)', 'Length (feet)','Time','Speed','Drop (feet)']
new_df = getNum(new_df,List,Type='int')

D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\ruanjian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


#### 到此数据补充完毕¶

In [14]:
new_df.shape

(290, 10)

In [15]:
stand_sdf = new_df[['Height (feet)','Speed','Length (feet)','Number of Inversions','Drop (feet)','Time','Vertical Angle (degrees)','G Force']]

In [16]:
def getStand(df):
    m = np.array(df.min().values)
    M = np.array(df.max().values)
    fenZi = M - m
    df = (df - m)/fenZi
    return df

In [17]:
stand_sdf  = getStand(stand_sdf)

In [18]:
#导入区间标准数据
stand = pd.read_excel('数值型标准.xlsx',index_col=1)

In [19]:
stand = stand.T.iloc[1:,4:]
# 处理时间
for i in range(0,8):
    stand['Duration (min:sec)'][i] = stand['Duration (min:sec)'][i].hour*60 +  stand['Duration (min:sec)'][i].minute
stand = stand.astype(float)

In [20]:
stand = stand.iloc[0:9,:]

In [21]:
stand  = getStand(stand) 

In [22]:
# 计算贝塔
a1=np.array(stand.iloc[0,:].values);
b1=np.array(stand.iloc[1,:].values);
beiTa = (b1-a1)/2

In [23]:
beiTa

array([0.48657503, 0.43162393, 0.3328908 , 0.5       , 0.4692492 ,
       0.30266667, 0.35185185, 0.5       ])

In [24]:
stand.index = ['惊悚型下限','惊悚型上限','刺激型上限','刺激型下限','家庭型上限','家庭型下限','儿童型上限','儿童型下限']

In [25]:
stand.T.to_csv('标准化.csv',index=0)

In [26]:
import math
x = stand.iloc[6,:].values
W = -(x-beiTa)/ math.sqrt(-math.log(0.1))

In [27]:
W

array([0.32065792, 0.28444459, 0.21937844, 0.32950511, 0.30924002,
       0.19946043, 0.23187397, 0.32950511])

In [28]:
# 计算各项得分
def getFen(df):
    df_fen = df*W
    return df_fen
# 计算总分
def getScore(df_fen):
    List = []
    for i in range(0,290):
        t = 0
        for x in range(0,8):
            t += df_fen.iloc[i,x]
        List.append(t)
    df_fen['Sum'] = List
    return df_fen
def getSortAndName(df_fen):
    # 排名降序
    df_fen = df_fen.sort_values('Sum',ascending=False)
    idx = [i for i in range(1,291)]
    df_fen['Rank'] = idx
    # 把名字弄上去
    df_fen['Name'] = df.iloc[df_fen.index,:1]
    return df_fen
# 给一份标准化的数据进行完整处理
def SolveOne(df):
     # 获取每一个得分
    d_fen = getFen(df)
    # 计算总得分
    d_fen = getScore(d_fen)
    # 计算排名
    d_fen = getSortAndName(d_fen)
    return d_fen

In [33]:
df_fen = SolveOne(stand_sdf)
df_fen 

,Height (feet),Speed,Length (feet),Number of Inversions,Drop (feet),Time,Vertical Angle (degrees),G Force,Sum,Rank,Name
299,0.320658,0.234884,0.088122,0.000000,0.309240,0,0.137294,0.205941,1.296138,1,Kingda Ka
292,0.211030,0.152675,0.193667,0.000000,0.215914,0.0997302,0.106784,0.308911,1.288712,2,Millennium Force
294,0.222293,0.157372,0.193880,0.000000,0.231732,0.160281,0.109835,0.205941,1.281334,3,Fury 325
167,0.167929,0.141400,0.137880,0.023536,0.191397,0.19946,0.122039,0.288317,1.271959,4,Hyperion
298,0.293626,0.216093,0.078469,0.000000,0.295004,0.0017809,0.137294,0.205941,1.228208,5,Top Thrill Dragster
291,0.208026,0.150326,0.160003,0.000000,0.220660,0.160281,0.106784,0.205941,1.212021,6,Leviathan
160,0.134440,0.098651,0.076496,0.094144,0.153434,0.108635,0.216619,0.308911,1.191330,7,Cannibal
285,0.160571,0.132709,0.148438,0.000000,0.181036,0.19946,0.122039,0.205941,1.150194,8,Coaster Through the Clouds
297,0.289872,0.169117,0.030963,0.000000,0.238139,0,0.137294,0.257426,1.122809,9,Superman: Escape from Krypton
165,0.158093,0.119556,0.116468,0.023536,0.152485,0.19946,0.122039,0.205941,1.097577,10,Schwur des Kärnan


In [34]:
df_fen.to_excel('最终排名.xlsx')

In [35]:
stand_sdf.index

Int64Index([  1,   3,   4,   5,   6,   7,   8,   9,  10,  12,
            ...
            289, 290, 291, 292, 294, 295, 296, 297, 298, 299],
           dtype='int64', length=290)

In [119]:
# 20%
# Data：源数据，调整d
# 返回调整后的排名浮动
def getLMD(df,d):
    t = df.copy()
    x = df.copy()
    df_fen = SolveOne(df)
    ANS = []
    for i in range(0,290): # 行
        List = []
        for j in range(0,8):
            # 某一个指标增加d
            x.iloc[i,j] = t.iloc[i,j]*(1+d)
            # 处理一份数据
            d_fen = SolveOne(x)
            List.append(d_fen.loc[x.index[i],'Rank'] - df_fen.loc[x.index[i],'Rank'])
            # 复原
            x.iloc[i,j] = t.iloc[i,j]
        ANS.append(List)
    return ANS

In [120]:
ANS20 = getLMD(stand_sdf,0.2) 
ANS_20 = getLMD(stand_sdf,-0.2) 
ANS10 = getLMD(stand_sdf,0.1) 
ANS_10 = getLMD(stand_sdf,-0.1) 

In [142]:
# 获取平均浮动值
def getFudong(a,fudong):
    ret = np.array(a[0])
    for i in range(1,len(a)):
        ret +=  np.array(a[i])
    return ret/len(a)

In [143]:
a2 = getFudong(ANS20,0.2)
a_2 = getFudong(ANS_20,0.2)
a1 = getFudong(ANS10,0.1)
a_1 = getFudong(ANS_10,0.1)

In [144]:
from pandas import Series, DataFrame
LMD = DataFrame()
LMD['+20%'] = a2
LMD['-20%'] = a_2
LMD['+10%'] = a1
LMD['-10%'] = a_1
LMD.index = stand_sdf.columns

In [146]:
LMD

,+20%,-20%,+10%,-10%
Height (feet),-5.927586,6.220690,-2.993103,3.020690
Speed,-5.534483,5.744828,-2.772414,2.751724
Length (feet),-6.903448,7.013793,-3.448276,3.431034
Number of Inversions,-4.755172,5.110345,-2.379310,2.665517
Drop (feet),-6.417241,6.517241,-3.275862,3.320690
Time,-6.848276,6.727586,-3.393103,3.324138
Vertical Angle (degrees),-5.317241,5.331034,-2.603448,2.634483
G Force,-15.972414,16.375862,-8.168966,8.324138


In [147]:
LMD.to_excel('灵敏度分析.xlsx')